# Faster RCNN Sagemaker Tutorial

This tutorial walks through the entire process of training a faster RCNN model on Sagemaker. There are six components that can be applied to most models trained on Sagemaker. This tutorial does not go into the logic of the faster RCNN model itself. Rather, we focus only on setting up and training the model. For a detailed explanation of faster RCNN, see [Faster R-CNN: Down the rabbit hole of modern object detection](https://tryolabs.com/blog/2018/01/18/faster-r-cnn-down-the-rabbit-hole-of-modern-object-detection/) from Tryo Labs.

- Notebook instance creation
- Data download and prep
- Docker container
- Hyperparameter settings
- Launching a model
- Monitoring

### Notebook instance creation

We recommend running this tutorial on a Sagemaker notebook instance. This will ensure you have proper access S3 and the AWS Elastic Container Registry. From the AWS console page, first go to S3 and create a new bucket if you don't already have one for sagemaker. With standard settings Sagemaker will only connect to buckets with `sagemaker` in the name, so we recommend using something like `mybucket-sagemaker`.

Next, go to the AWS Sagemaker and select `Notebook instance`. Select `Create notebook instance`, give your instance any name you like, and select any instance type you want. You don't need a powerful notebook instance, since we'll be doing the training on different instances. But sometimes you might want to experiment with some training on the notebook instance itself, so more powerful instances are available. For now, an ml.m5.2xlarge should be fine. Select additional configuration and increase your volume size to 100GB. We need the extra size in order to get the model data, and build our docker containers. Under `Permissions and Encryption` select `Create a new role`. A new window will pop up. Leave this with the default settings, and click `Create role`. Everything else can be left as default. Click `Create notebook instance`. 

It will take a few minutes to create the notebook instance. While that's happening, we need to add one more permission to our IAM role we just created. On the `Notebook instances` page you will see your new instance. Click the instance name link, scroll down to `Permissions and encryption` and click the link under `IAM role ARN`. On the summary page, select `Attach policies` in the filter policies box search for `container registry`, check the box next to `AmazonEC2ContainerRegistryFullAccess`, and click `Attach policy`. This allows us to store our new container image we want to use with Sagemaker.

If you don't already have a container repository on Elastic Container Registry, got to `Elastic Container Registry` on the AWS dashboard, click `create repository` and pick any name you like.

### Data download and prep
The script below contains everything to download the COCO 2017 dataset, as well as resnet weights pretrained on imagenet data. These are the same weights included in the Keras package. We download them manually for stability, because they are sometimes retrained between versions of Keras. The script will download everything to the Sagemaker notebook instance, assemble everything in the file structure the faster RCNN model expects to see, archives it, and saves it to your S3 bucket.

Once your instance is ready, click `Open JupyterLab`. When the launcher appears, select Terminal at the bottom of the page and run the following commands:

```
S3_BUCKET=[name of your s3 bucket not including s3://]

############################################################
# Clone repo
############################################################

cd ~/SageMaker
git clone https://github.com/aws-samples/deep-learning-models
BASE_DIR=$HOME/SageMaker
mkdir -p $BASE_DIR/data/coco
mkdir -p $BASE_DIR/data/weights

############################################################
# Download all data files
############################################################

wget -O $BASE_DIR/data/coco/train2017.zip http://images.cocodataset.org/zips/train2017.zip
wget -O $BASE_DIR/data/coco/val2017.zip http://images.cocodataset.org/zips/val2017.zip
wget -O $BASE_DIR/data/coco/annotations_trainval2017.zip \
        http://images.cocodataset.org/annotations/annotations_trainval2017.zip
wget -O $BASE_DIR/data/weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5 \
        https://github.com/keras-team/keras-applications/releases/download/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5

############################################################
# Decompress arrange and archive
############################################################

unzip -q $BASE_DIR/data/coco/train2017.zip -d $BASE_DIR/data/coco
unzip -q $BASE_DIR/data/coco/val2017.zip -d $BASE_DIR/data/coco
unzip $BASE_DIR/data/coco/annotations_trainval2017.zip -d $BASE_DIR/data/coco
rm $BASE_DIR/data/coco/*.zip
cd $BASE_DIR/data/
tar -cf coco.tar coco

############################################################
# Upload to S3
############################################################

aws s3 cp $BASE_DIR/data/coco.tar s3://${S3_BUCKET}/faster-rcnn/data/coco/coco.tar
aws s3 cp $BASE_DIR/data/weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    s3://${S3_BUCKET}/faster-rcnn/data/weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
    
```

This will take about 5 minutes.

### Docker container

While the data prep script is running, you can open another terminal by clicking the `+` in the upper left corner of your screen and starting a new terminal. First, a little clarification on how Sagemaker works. When you launch a Sagemaker training job, each instance on that job will download a Docker image from ECR, setup a series of environment variables so it knows your hyperparameters and where to find the data, and launches your training script. Most of the time, you can use Sagemaker's built in containers, but for more complex models you might want to create your own. 

Under `deep-learning-models/models/vision/detection/docker` you'll find the Dockerfile that create the image we use for training. A detailed description of that file, and how to create your own, is included in the readme. Below are the basic commands to build this image and upload it to ECR.

```
ECR_REPO=[the ECR repo created earlier]
cd ~/SageMaker/deep-learning-models/models/vision/detection/docker
ALGO=frcnn-tutorial
docker build -t ${ECR_REPO}/${ALGO} -f Dockerfile.frcnn .

# login to ECR
REGION=$(aws configure get region)
ACCOUNT=$(aws sts get-caller-identity --query Account --output text)
FULLNAME="${ACCOUNT}.dkr.ecr.${REGION}.amazonaws.com/${ECR_REPO}:${ALGO}"
$(aws ecr get-login --region ${REGION} --no-include-email)

# push image to ECR
docker tag ${ECR_REPO}/${ALGO} ${FULLNAME}
docker push ${FULLNAME}
echo ${FULLNAME}
```

### Hyperparameter settings

Now we can start setting up our model. Set the variables in the paragraph below to access your S3 bucket and ECR repo.

Below, enter your s3 bucket, image name, and a user name.

After this paragraph, you don't need to modify anything to run training with the standard hyperparameters.

In [1]:
s3_bucket = "jbsnyder-sagemaker" # name of your s3 bucket without s3://
docker_image = "578276202366.dkr.ecr.us-east-1.amazonaws.com/jbsnyder:frcnn-tutorial" # the output of `echo ${FULLNAME}` from the previous section something like 12345.dkr.ecr.us-east-1.amazonaws.com/name:algo
user_id = "jbsnyder"# this can be anything you like, used for keeping track of your training jobs

In [2]:
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
from datetime import datetime
import os
import pprint

Below are the common hyperparameters to tune. All of these hyperparameters are passed to the `deep-learning-models/models/vision/detection/tools/train_sagemaker.py`. More customization can be done in the `train_sagemaker` script. All lines referenced below refer to this file.

`schedule` determines the learning rate schedule. `1x` performs a 10x drop in the learning rate at the 8th and 10th epochs. The other option currently included is `cosine` which performs a cosine decay with restarts. These schedulers are lines 112-121.

`fp16` refers to whether a model should train using mixed precision. Mixed precision training is supported on p3 and g4 instances.

`base_learning_rate` this is the learning rate after the warmup period.

`warmup_steps` number of steps for warmup.

`warmup_init_lr_scale` the scale of the warmup start level. 3.0 means the learning rate starts as 1/3 of the base_learning_rate value.

`batch_size_per_device` number of images per GPU.

`instance_type` the type of EC2 instance for training. For highest performance use `ml.p3dn.24xlarge`. The `ml.p3.16xlarge` and `ml.p3.8xlarge` are slightly lower performance while being more cost effective. The `g4dn.12xlarge` instance type is even lower cost, but much slower than the p3 instances.

`instance_count` number of instances for training.

`num_workers_per_host` number of training jobs per instance. This should be the number of GPUs on the instance, so should be set to 8 for `ml.p3dn.24xlarge` or `ml.p3.16xlarge` instances, and 4 for `ml.p3.8xlarge` or `g4dn.12xlarge`.

`use_conv` the final bounding box head of the faster RCNN model can use convolutions instead of fully connected layers. We observe slightly better performance with convolution layers, but provide an option to run either.

`use_rcnn_bn` batch normalization of the bounding box head layers. This is only useful when using fully connected layers, and will be ignored if `use_conv` is set to True.

`ls` label smoothing. In some cases, faster rcnn performance is hurt by a large imbalance in the number of positive versus negative regions. Label smoothing can help mitigate this.

In [3]:
hyperparameters = {
    'schedule': '1x',
    'fp16': True,
    'base_learning_rate': 15e-3,
    'warmup_steps': 500,
    'warmup_init_lr_scale': 3.0,
    'batch_size_per_device': 4,
    'instance_type': 'ml.p3dn.24xlarge',
    'instance_count': 1,
    'num_workers_per_host': 8,
    'use_conv': True,
    'use_rcnn_bn': False,
    'ls': 0.0,
}

The model configuration file also contains additional less commonly adjusted hyperparamters, as well as data pipeline settings.

In [4]:
model_cfg = "../configs/sagemaker_default_config.py"

Now we can setup our training job. We first get the Sagemaker execution role, so the training instances now which account they're connected to. We add a timestamp to keep track of multiple training jobs. And finally give Sagemaker the path to our model training files.

In [5]:
role = get_execution_role()
now = datetime.now()
time_str = now.strftime("%d-%m-%Y-%H-%M")
date_str = now.strftime("%d-%m-%Y")
source_dir = ".."
main_script = "tools/train_sagemaker.py"

Fill in details about our training instances. Get the instance types, GPUs, and number of instances. We also setup any options we want for MPI. 

In [6]:
hvd_processes_per_host = hyperparameters['num_workers_per_host']
hvd_instance_type = hyperparameters['instance_type']
hvd_instance_count = hyperparameters['instance_count']

distributions = {
    "mpi": {
        "enabled": True,
        "processes_per_host": hvd_processes_per_host,
        "custom_mpi_options": "-x OMPI_MCA_btl_vader_single_copy_mechanism=none -x TF_CUDNN_USE_AUTOTUNE=0"
    }
}

Setup the output paths on S3. This is where the final model and training logs will be stored.

Name the training job using user name and time stamp.

Create the training configuration.

In [7]:
s3_path = os.path.join('s3://{}/faster-rcnn/outputs/{}'.format(s3_bucket, date_str))

job_name = '{}-frcnn-{}'.format(user_id, time_str)

output_path = os.path.join(s3_path, "output", job_name)

configuration = {
    'configuration': 'configs/sagemaker_default_model_config.py', 
    's3_path': s3_path,
    'instance_name': job_name
}
configuration.update(hyperparameters)

Lastly, we need to set our training channels. Channels are environmental variables in the Sagemaker training instances that tell Sagemaker where to find data. Each channel has an S3 location that will copy to the instance. For example, if we pass a dictionary with

```
{"coco": "s3://my-bucket/data/coco_data/",
 "weights": "s3://my-bucket/data/resnet_weights/"}
```

Sagemaker will create two directories on each training instance

```
/opt/ml/input/data/coco/
/opt/ml/input/data/weights/
```

and copy all contents of the respective S3 locations to these directories. Addtionally, each training instance will get two environmental variables

```
SM_CHANNEL_COCO
SM_CHANNEL_WEIGHTS
```

that are set to these directories.

So we want to create a dictionary that points to where we uploaded the data at the beginning of this tutorial.

In [8]:
channels = {
    'coco': 's3://{}/faster-rcnn/data/coco/'.format(s3_bucket),
    'weights': 's3://{}/faster-rcnn/data/weights/'.format(s3_bucket)
}

New we can create our Sagemaker estimator, and launch training.

In [9]:
estimator = TensorFlow(
                entry_point=main_script, 
                source_dir=source_dir, 
                image_name=docker_image, 
                role=role,
                framework_version="2.1.0",
                py_version="py3",
                train_instance_count=hvd_instance_count,
                train_instance_type=hvd_instance_type,
                distributions=distributions,
                output_path=output_path, train_volume_size=200,
                hyperparameters=configuration
)

And finally, launch training.

In [10]:
estimator.fit(channels, wait=False, job_name=job_name)

Since we set `wait=False` it will seem like nothing is happening. Since training take about 6 hours on a p3.16xlarge, we don't want our notebook to have to wait on it.

If you go back to the Sagemaker homepage and click `Training Jobs`, you'll now see a new training job with a name like `user-frcnn-12-27-2020-4-29`. You can click the link to see what your training job is doing. The training will take about 5-10 minutes to set up, depending on your instance type (instances with NVME storage, p3dn and g4dn, will start faster). It first downloads your data from S3, then your Docker image from ECR. At this point, you have a few options for monitoring training. If you scroll down on this page, there is a section called `Monitor`. Here you can see your CPU and GPU usage, or view the logs that each instance is outputing. 

A more fun way to monitor is using Tensorboard. Once training starts, you'll find a new set of directories in your S3 bucket in the form

```
s3://[your bucket]/faster-rcnn/
```

In a terminal on your notebook instance, run

```
S3_BUCKET=[your bucket]
source activate tensorflow_p36
tensorboard --logdir s3://${S3_BUCKET}/faster-rcnn/outputs/
```

In your browser, go to 

`https://[your notebook instance name].notebook.us-east-1.sagemaker.aws/proxy/6006/`

You might see a `No dashboards are active for the current data set` for a few seconds while Tensorboard loads data from S3. Once data is loaded, you should see a dashboard containing your training progress. Under the Scalars section you can see model loss and MaP performance. Under Images you can see how well you model is currently doing on random COCO images.

### Scalars

![SCALARS](assets/scalars.png)

### Images

![IMAGE](assets/image.png)